# Master TIDE - Conférences Python 2020

Francis Wolinski

&copy; 2020 Yotta Conseil

# 2. Pandas : accès aux données

In [ ]:
# import des modules usuels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# commande magique pour l'affichage des graphiques
%matplotlib inline

# options d'affichage
pd.set_option("display.max_rows", 16)
plt.style.use('seaborn-darkgrid')

In [ ]:
# chargement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo.head()

In [ ]:
# index
geo.index

In [ ]:
# colonnes
geo.columns

Point de vocabulaire :
- Les objets *Series* et *DataFrame* possèdent des *index* qui peuvent être des entiers (par défaut) ou bien des labels (chaines de caractères, ou encore des tuples pour les index multiples / hiérarchiques). Lorsque l'index est unique, ils ont un comportement de type dictionnaire auquel on accède au moyen d'une clé.
- Ces objets sont aussi basés sur des tableaux (*ndarray*) auxquels on peut accéder par position, c'est-à-dire par un entier appelé *indice* (en démarrant à 0). Ils ont un comportement de type séquence auquel on accède par un entier.

## 1.1 Rappel sur les accès par indices en Python



Notation|Signification
-|-
s[i]|accès à l'élement situé à la position i (démarrant à 0)
s[i:j]|accès aux élements situés entre la position i et la position j-1
s[i:j:k]|accès aux élements situés entre la position i et la position j-1 par pas de k
s[:i]| accès à tous les éléments jusqu'à la position i-1
s[i:]|accès à tous les éléments à partir de la position i
s[:]|accès à tous les éléments
s[::-1]|accès à tous les éléments dans l'ordre inverse

**Rappel** : en Python le premier indice d'une séquence de longueur L est 0 et le dernier est L - 1. Par commodité, une séquence peut aussi être accédée par des indices négatifs allant de -L à -1.

P|y|t|h|o|n
-|-|-|-|-|-
0|1|2|3|4|5
-6|-5|-4|-3|-2|-1

**Remarques importantes**

Cette notation utilisée pour les indices a été reprise pour les index des objets *Series* et *dataFrame*.
- Lorsque 2 index sont indiqués, le second index est inclus dans la sélection, contrairement aux indices (idem avec les colonnes).
- Lorsque 2 index sont indiqués, la notation fonctionne uniquement lorsque l'index est unique (idem avec les colonnes).

## 1.2 *Series*

Il existe plusieurs opérateurs pour effectuer des sélections dans les objets *Series* :
+ `[]` : sélection à partir d'un index ou d'un indice (si l'index n'est pas un entier) :
    - un seul index ou indice : `s[i]`
    - liste d'index ou d'indices : `s[[i, j, k]]` (fancy indexing)
    - plage d'index ou d'indices : `s[i:j]` et `s[i:j:k]` (attention index vs indices)
    - vecteur de booléens (masque) : `True` = sélection, `False` = non sélection
+ `.loc[]` idem mais réservé aux index
+ `.iloc[]` idem mais réservé aux indices

Lorsque la sélection correspond à une seule valeur, l'opération retourne un scalaire.
Lorsque la sélection correspond à plusieurs valeurs, l'opération retourne un objet `Series`.

N.B. : Les instructions de sélection avec les opérateurs `loc` et `iloc` permettent aussi l'affectation de valeurs.

### 1.2.1 Sélection par index ou indice

In [ ]:
# sélection par index
s = geo['Commune']
s.loc['75113'] # ou s['75113']

In [ ]:
# sélection par indice
s.iloc[30836] # ou s[30836]

In [ ]:
# sélection par une liste d'index
s.loc[['01001', '01004', '01006']] # ou s[['01001', '01004', '01006']]

In [ ]:
# sélection par une liste d'index
l = ['01001', '01004', '01006']
s.loc[l] # ou s[['01001', '01004', '01006']]

In [ ]:
# sélection par une liste d'indices
s.iloc[[0, 2, 4]] # ou s[[0, 2, 4]]

In [ ]:
# sélection par une plage d'index
# 01006 est inclus
s.loc['01001':'01006'] # ou s['01001':'01006']

In [ ]:
# sélection par une plage d'indices
# 5 est exclu
s.iloc[0:5] # ou s[0:5]

In [ ]:
# sélection par une plage d'index
# 01006 est inclus
s.loc['01001':'01006':2] # ou s['01001':'01006':2]

In [ ]:
# sélection par une plage d'indices
# 5 est exclu
s.iloc[0:5:2] # ou s[0:5:2]

### 2.2.1 Sélection avec un masque booléen

Il est possible de sélectionner une partie d'un objet `Series` à partir d'un masque booléen, i.e. un autre objet `Series` de même dimension et composé de booléens.

Opérateurs logiques:
- & (ET)
- | (OU)
- ~ (NON)

In [ ]:
# Series
s

In [ ]:
# vecteur de booléens selon si le nom de la ville commence par "A"
masque = s.str.startswith("A")
masque

On obtient 2 objets `Series` parallèles et on ne retient que les élements qui correspondent à un `True`.

In [ ]:
# sélection à partir du vecteur de booléens
s.loc[masque]  # ou bien s[masque]

In [ ]:
s.str.len()

In [ ]:
# les communes qui commencent par SAINT
s.loc[s.str.startswith("SAINT")]

<div class="alert alert-success">
<b>Exercice 1</b>
<ul>
    <li>Quelles communes commencent par la lettre "Z" ?</li>
    <li>Quelles communes commencent et finissent par la lettre "Y" ?</li>
    <li>Combien de communes sont "SUR-MER" ?</li>
    <li>Quelles communes ont un nom avec la préposition "SOUS" ?</li>
    <li>Quelles communes ont un nom avec "SOUS" mais pas la préposition "SOUS" ?</li>
</ul>
</div>

#### Point sur les expressions régulières

In [ ]:
# les communes comportant 2 Z
s.loc[s.str.contains("Z.*Z")].head(16)

Les **expressions régulières**, ou **regex**, permettent de filtrer des chaines de caractères selon un motif. Dans ces expressions, certains caractères ont une signification spéciale pour permettre le filtrage :

- `.`: tout caractère
- `^`: le début de la chaine
- `$`: la fin de la chaine
- `*`: 0 ou plusieurs répétitions du motif précédent
- `+`: 1 ou plusieurs répétitions du motif précédent
- `?`: 0 ou 1 répétition du motif précédent
- `\`: pour que ces caractères soient traités normalement, il faut les préfixer pat un `\`.

La méthode `contains()` prend des expressions régulières par défaut. Il faut utiliser l'option `regex=False` pour chercher des chaines normales.

## 2.3 *DataFrame*

Les mêmes opérateurs existent pour effectuer des sélections dans les objets de type *DataFrame* :
+ `[]` : sélection à partir d'un index ou d'un indice (si l'index n'est pas un entier) :
    - une seule colonne : df[a]
    - liste de colonnes ou d'indices : df[[a, b, c]] (fancy indexing)
    - plage d'index ou d'indices : df[i:j] et df[i:j:k] (Attention index vs indices)
    - vecteur de booléens (masque) : *True* = sélection, *False* = non sélection
+ `.loc[]` idem mais réservé aux index
+ `.iloc[]` idem mais réservé aux indices

Lorsque la sélection correspond à une seule valeur, l'opération retourne un scalaire.
Lorsque la sélection correspond à une colonne ou à une ligne, l'opération retourne un objet `Series`.
Lorsque la sélection correspond à plusieurs colonnes et à plusieurs lignes, l'opération retourne un objet `DataFrame`.

N.B. : Les instructions de sélection avec les opérateurs `loc` et `iloc` permettent aussi l'affectation de valeurs.

### 2.3.1 Sélection de colonnes

En sélectionnant une seule colonne, on obtient un objet `Series` qui partage l'index du `DataFrame` initial. Cf. ci-dessus.

En sélectionnant plusieurs colonnes, on obtient un nouveau `DataFrame` sous-ensemble du `DataFrame` initial et partageant son index avec autant de lignes que le `DataFrame` initial.

In [ ]:
# sélection d'une colonne
geo["Commune"]

In [ ]:
# communes + superficie et population
l = ["Commune", "Superficie", "Population"]
geo[l].head()

In [ ]:
# communes + superficie et population
geo[["Commune", "Superficie", "Population"]].head()

### 2.3.2 Sélection de lignes

En sélectionnant une seule ligne, on obtient un objet `Series` dont l'index correspond aux colonnes du `DataFrame` initial.

En sélectionnant plusieurs lignes, on obtient un nouveau `DataFrame` sous-ensemble du `DataFrame` initial et partageant son index avec autant de colonnes que le `DataFrame` initial.

In [ ]:
# sélection d'une ligne
s = geo.loc["01001"]
s

In [ ]:
type(s)

In [ ]:
# sélection d'une ligne
geo.iloc[0]

In [ ]:
# sélection de plusieurs lignes
geo.loc["01001":"01006"]

In [ ]:
# sélection de plusieurs lignes
geo.iloc[0:5]

### 2.3.3 Sélection de lignes et de colonnes

En sélectionnant plusieurs lignes et plusieurs colonnes, on obtient un nouveau `DataFrame` sous-ensemble du `DataFrame` initial.

In [ ]:
# sélection de plusieurs lignes et de plusieurs colonnes
geo.loc["01001":"01006", "Commune":"Statut"]

In [ ]:
# sélection de plusieurs lignes et de plusieurs colonnes
geo.iloc[0:5, 1:5]

### 2.3.4 Sélection avec des conditions portant sur les valeurs

Il est également possible de sélectionner une partie d'un `DataFrame` à partir de conditions portant sur les valeurs.

La conjonction de conditions est notée `&`, la disjonction est notée `|` et la négation `~`.

In [ ]:
# les communes des Hauts-de-Seine avec une population supérieure à 80000
geo.loc[(geo["Département"] == "HAUTS-DE-SEINE") & (geo["Population"] > 80.0)]

<div class="alert alert-success">
<b>Exercice 2</b>
<ul>
    <li>Affichez l'altitude moyenne, la superficie et la population de toutes les communes dénommées Sainte-Colombe.</li>
    <li>Combien y a-t-il de départements dans lesquels au moins une commune a plus de 200.000 habitants ?</li>
    <li>Donner la liste de ces départements.</li>
    <li>Combien y a-t-il de communes dont le nom comporte 2 lettres?</li>
    <li>Donnez la liste de ces communes dans l'ordre alphabétique.</li>
    <li>Sélectionnez les communes pour lesquelles l'atitude moyenne vaut la superficie.</li>
    <li>Quelle est la population totale de ces communes ?</li>
    <li>Y a-t-il des communes au niveau de la mer ?</li>
</ul>
</div>

### Index non unique

In [ ]:
# test de l'unicité de l'index
geo.index.is_unique

In [ ]:
# reset de l'index
geo = geo.reset_index()
geo = geo.set_index('Commune', drop=True)
geo.head()

In [ ]:
# test de l'unicité de l'index
geo.index.is_unique

In [ ]:
# accès avec un index non unique
geo.loc['SAINTE-COLOMBE']

In [ ]:
# tête
geo.head()

In [ ]:
# tentative d'accès à une plage d'index
geo.loc["L'ABERGEMENT-CLEMENCIAT":"AMBLEON"]

### latitude et longitude

In [ ]:
# colonne "geo_point_2d"
geo["geo_point_2d"]

In [ ]:
# accès à une valeur "geo_point_2d"
geo.loc["L'ABERGEMENT-CLEMENCIAT", "geo_point_2d"]

In [ ]:
geo["Latitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[0]))
geo["Longitude"] = geo["geo_point_2d"].apply(lambda x: float(x.split(', ')[1]))
geo.head()

In [ ]:
# avec la méthode extract()
geo["geo_point_2d"].str.extract("(.*), (.*)", expand=True)

In [ ]:
# avec la méthode extract()
geo[["Latitude" , "Longitude"]] = geo["geo_point_2d"].str.extract("(.*), (.*)", expand=True)
geo